# OpenIntro Stats Exercises

The following is a series of exercises, taken from OpenIntro Stats 4ed, for the purposes of developing my own understanding of the statistical tests alongside increasing familiarity with common statistical packages. A publicly available copy of OpenIntro Stats can be found here: https://www.openintro.org/go/?id=os4_for_screen_reader and is required to understand the context of many of these questions.

In [43]:
from scipy import stats
import numpy as np
import pandas as pd
import seaborn as sns

## Chpt 6

### 6.2 Hypothesis Tests for the Difference of Two Sample Means

#### 6.13 Guided Practice

In [2]:
# Create the dataframe given in the question

df = pd.DataFrame({'heart_attack' : [145, 200], 'no_event' : [12788, 12738]} , index = ['fish_oil', 'placebo'])
df['total'] = df['heart_attack'] + df['no_event']
df


,heart_attack,no_event,total
fish_oil,145,12788,12933
placebo,200,12738,12938


First verify that a normal approximation is valid. Each subject was randomly assigned to one of the two groups so we may assume independence holds. Now we verify the success-failure condition:

In [3]:
# Fish oil group:

p1_hat , p2_hat = df['heart_attack'] / df['total'] 
n1, n2 = df['total']

all([n1*p1_hat>10, n1*(1-p1_hat)>10, n2*p2_hat>10, n2*(1-p2_hat)])

True

All good here. We are safe to use a normal distribution to approximate the sampling distribution. We now construct the 95% confidence interval.

In [7]:
point_estimate = p1_hat - p2_hat
std_error = np.sqrt( p1_hat*(1-p1_hat)/n1 + p2_hat*(1-p2_hat)/n2 )

# Compute corresponding z-value using an inverse CDF of a standard normal distribution:

z = stats.norm.ppf(0.975)

# Convert to float dtype for more aesthetic printing

print([float(point_estimate-z*std_error), float(point_estimate+z*std_error)])

[-0.007041644824162001, -0.0014517763930541822]


Observing the confidence interval, we have evidence to reject the hypothesis that the fish oil treatment does not affect heart attack outcomes.

#### 6.15 Guided Practice

Not exactly what Ex 6.15 asks for but a good exercise nonetheless.

In [8]:
# Create the data given:

df = pd.DataFrame({'Yes':[500,505], 'No':[44425,44405]} )
df.index = ['Mammogram','Control']
df

,Yes,No
Mammogram,500,44425
Control,505,44405


Let $p_1\, \, p_2$ be the true proportion of those who die of cancer in the mammogram and control groups respectively. We wish to test if there is a statistically significant difference between the two proportions. We state our null and alternate hypotheses as:

$
H_0 : p_1 - p_2 = 0 
$

$
H_1 : p_1 - p_2 \neq 0
$

We will conduct a two-tailed test at the 5% significance level. We first verify that a normal distribution is a valid approxiation of the sample distribution in this case:

In [9]:
n1, n2 = df['Yes'] + df['No']
p1, p2 = df['Yes'] / [n1, n2]

all([n1*p1>10, n1*(1-p1)>10, n2*p2>10, n2*(1-p2)>10])

True

The sample parameters seem reasonable to use a normal distribution as an approximation. Since the women were randomly assigned to one of the two groups, we may assume independence holds both within each group and across each group.

We now compute the p-value for our observed statistic:

In [126]:
mu = 0

# Estimate the standard deviation of the null distribution using the sample parameters
sigma = np.sqrt( p1*(1-p1)/n1 + p2*(1-p2)/n2 )

# p1-p2<0 so we compute the p-value using symmetries of the normal distribution

p = 2*stats.norm(mu, sigma).cdf(p1-p2)
float(p)

0.8697840130334852

This is clearly well above the 5% threshold so we have no evidence to reject the null hypothesis.

#### 6.27 Exercise

In [11]:
# Create the dataframe given in the question

df = pd.DataFrame({'Control':[35, 193, 64], 'Pilots': [19, 132, 51], 'Truck Drivers': [35, 117, 51], 'Train Operators':[29, 119, 32], 'Bus/Taxi/Limo Drivers':[21, 131, 58]})
df.index = {'Less than 6 hours of sleep':0, '6 to 8 hours of sleep':1, 'More than 8 hours':2}
df.loc['Total'] = df.sum()
df

,Control,Pilots,Truck Drivers,Train Operators,Bus/Taxi/Limo Drivers
Less than 6 hours of sleep,35,19,35,29,21
6 to 8 hours of sleep,193,132,117,119,131
More than 8 hours,64,51,51,32,58
Total,292,202,203,180,210


We wish to conduct a hypothesis test as to whether or not there is a difference between the proportion of truck drivers who get less than 6 hours of sleep per day and the corresponding proportion for non-transportation workers (i.e. the control group).

We begin by setting up our hypotheses. Let $\hat{p_t}, \, \hat{p_c}$ represent the sample proportions for the truck drivers and control groups respectively and $p_t, \, p_c$ be the corresponding population parameters. Then, our hypotheses are:

$H_0: p_t-p_c=0$

$H_1: p_t \neq p_c$

We will now verify that the success-failure condition holds to ensure that a normal distribution is a valid approximation of the sample distribution by using the pooled proportion.

In [77]:
n_c, n_t = df.loc['Total', ['Control', 'Truck Drivers']]
n = n_c + n_t
c, t = df.loc['Less than 6 hours of sleep', ['Control', 'Truck Drivers']] 
p_pooled = (c+t)/n
all( [ n*p_pooled > 10, n*(1-p)>10 ] )

True

We appear to have a sufficient sample size to make a normal distribution valid here. Since our null hypothesis is that there is no difference in the means between the two groups, we will compute the pooled standard error and use this as the standard deviation of our null distribution.

In [88]:
p_c, p_t = c/n_c, t/n_t
SE_pool = np.sqrt( p_pooled*(1-p_pooled)/n_t + p_pooled*(1-p_pooled)/n_c )
SE_pool

np.float64(0.031842080861455284)

We now compute the test statistic. We will use a 5% significance level.

In [92]:
# Double to get the p-value since the CDF function only gives the probability in one-tail of the normal distribution
p = 2*(1-stats.norm(0, SE_pool).cdf(p_t-p_c))
p

np.float64(0.09887007932782055)

This is clearly outside of our 5% significance level, so we do not have sufficient evidence to reject the null hypothesis.

### 6.4 Testing for Independence in Two-Way Tables (Chi-Squared Tests)

#### 6.37 Exercise

Determine if their is a statistically significant difference in the response of graduates vs non-graduates in support for off-shore drilling.

We begin by creating the data.

In [39]:
df = pd.DataFrame({'Yes' : [154, 180, 104] , 'No' : [132, 126, 131]})
df.index = ['Support', 'Oppose', 'Do not know']
df.loc['Total'] = df.sum()
df.loc[:, 'Total'] = df.sum(axis=1)
df

,Yes,No,Total
Support,154,132,286
Oppose,180,126,306
Do not know,104,131,235
Total,438,389,827


And formulating our hypotheses:

$H_0:$ A student's support for off-shore drilling is independent of whether they are a graduate or not.

$H_1:$ A student's support for off-shore drilling is dependent on whether they are a graduate or not.

We will test this at the 5% significance level.

Now we compute the expected counts in a separate dataframe.

In [52]:
df_exp = df.copy()
total = df.loc['Total', 'Total']
x, y = df.shape
for i in range(x):
    for j in range(y):
        df_exp.iloc[i,j] = df.iloc[i,y-1] * df.iloc[x-1,j] / total
df_exp

C:\Users\ccbar\AppData\Local\Temp\ipykernel_17376\1736691997.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '151.4727932285369' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_exp.iloc[i,j] = df.iloc[i,y-1] * df.iloc[x-1,j] / total
C:\Users\ccbar\AppData\Local\Temp\ipykernel_17376\1736691997.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '134.5272067714631' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_exp.iloc[i,j] = df.iloc[i,y-1] * df.iloc[x-1,j] / total


,Yes,No,Total
Support,151.472793,134.527207,286
Oppose,162.065296,143.934704,306
Do not know,124.461911,110.538089,235
Total,438.000000,389.000000,827


All expected counts are above 5 making. The students were randomly selected so we may assume independence holds here. Now to compute the test statistic.

In [50]:
test_stat = ((df_exp - df)**2/df_exp).sum().sum()
test_stat

np.float64(11.460816627638106)

In [53]:
# Use x-2 and y-2 here because we wish to exclude the total rows and columns in the table
stats.chi2(df=(x-2)*(y-2)).sf(test_stat)

np.float64(0.003245751677744377)

Clearly this is less than the 5% mark providing us with evidence that a student's support for off-shore drilling is dependent on whether or not they are a graduate.

## Chapter 7: Inference for Numerical Data (t-tests and ANOVA)

### 7.3 Difference of Two Means

#### 7.25 Exercise

We are given the following data summarising emergency room admissions due to traffic accidents on Friday 13th compared with Friday 6th in a given month. We will conduct a hypothesis test at the 5% significance level as to whether or not there is a significant difference between the means and construct a confidence interval.

In [179]:
# Start by creating the data
df = pd.DataFrame({'6th':[7.5, 3.33, 6], '13th':[10.83,3.6,6]})
df.index=['Mean', 'SD', 'n']
df.loc[['Mean','SD'],'Difference'] = df.loc[['Mean','SD'],'6th'] - df.loc[['Mean','SD'],'13th']
df.loc['n', 'Difference'] = 6
df

,6th,13th,Difference
Mean,7.50,10.83,-3.33
SD,3.33,3.60,-0.27
n,6.00,6.00,6.00


We state our hypotheses. Let $x_6$, $x_{13}$ be the means for the number of emergency room admissions on Friday the 6th and 13th respectively. Then,

$H_0: x_6-x_{13}=0$

$H_1: x_6-x_{13}\neq 0$

Now we conduct the test.

In [183]:
# Compute the point estimate
p = df.loc['Mean', 'Difference']

# Compute the SE. We assume that the SD calculated in the table is in fact the sample standard deviation.
SE = np.sqrt((df.loc['SD', ['6th', '13th']]**2/df.loc['n']).sum())

# Find the p-value for our point estimate using df=5 since both samples had size 6
p_val = 2*stats.t(df=5, scale=SE).cdf(p)
p_val

np.float64(0.1571370854992739)

The p-value is substantially above the 5% threshold meaning we do not have sufficient evidence to reject the null, namely, there is no difference in emergency room admissions on Friday 13th vs Friday 6th.

We now construct the corresponding confidence interval.

In [119]:
# Get t-score
t = stats.t(df=5).ppf(0.975)

# Use float for a nicer display
(float(p-t*SE), float(p+t*SE))

(-8.476398566594813, 1.8163985665948124)

#### 7.26 Exercise

We are given the following data summarising the different mean prices between diamonds of different carats. We will construct a 95% confidence interval for the average difference between the standardised prices.

In [177]:
# Construct the data

df = pd.DataFrame({'0.99 Carats':[44.51, 13.32, 23], '1 Carat':[56.81, 16.13, 23]})
df.index = ['Mean', 'SD', 'n']
df

,0.99 Carats,1 Carat
Mean,44.51,56.81
SD,13.32,16.13
n,23.00,23.00


In [178]:
# Compute the point estimate
p = df.loc['Mean','1 Carat'] - df.loc['Mean', '0.99 Carats']

# Compute the standard error
SE = np.sqrt(( df.loc['SD']**2 / (df.loc['n']) ).sum())

# Compute the t-score using df=22 since both categories have a sample size of 23
t = stats.t(df=22).ppf(0.975)

# Display the confidence interval
# Use float() to improve aesthetics
(float(p-t*SE), float(p+t*SE))

(3.2540004246094245, 21.345999575390586)

Thus we have sufficient evidence to reject the hypothesis that 1 carat diamonds are different in price to 0.99 carat diamonds.

#### 7.27 Exercise

We wish to conduct a hypothesis test at the 5% significance level as to whether or not the mean weight of chickens fed linseed differs to those fed horsebean.

In [167]:
# Construct the data
df = pd.DataFrame({'Mean':[160.20, 218.75], 'SD':[38.63,52.24], 'n':[10,12]})
df.index=['horsebean', 'linseed']
df

,Mean,SD,n
horsebean,160.20,38.63,10
linseed,218.75,52.24,12


Our hypotheses are:

$H_0: x_h-x_l = 0$

$H_1: x_h-x_l \neq 0$

where $x_h, \, x_l$ are the mean weights of chickens fed horsebean and linseed respectively.

In [175]:
# Test statistic
p = df.loc['horsebean','Mean'] - df.loc['linseed', 'Mean']

# Compute standard error
SE = np.sqrt((df.loc[:, 'SD']**2/df.loc[:,'n']).sum())

# Compute p-value
p_val = 2*stats.t(df=9, scale=SE).cdf(p)
p_val

np.float64(0.014552317647771205)

In [165]:
p = 1.4
p_0 = 0.0
SE = 3.2

t_score = (p-p_0)/SE

stats.t(df=5).cdf(t_score)
stats.t(df=5, loc=p_0, scale=SE).cdf(p)

np.float64(0.6599995712862106)

#### 7.39 Exercise

We begin by recreating the data given in the text. The table gives the weekly METS (a measurement of physical activity) of people based on their coffee consumption.

In [97]:
df = pd.DataFrame({'1 cup or less/week':[18.7,21.1,12215], '2-6 cups/week':[19.6,25.5,6617], '1 cup/day':[19.3,22.5,17234],'2-3 cups/day':[19.9,22.0,12290], '4 or more cups/day':[17.5,22.0,2383]})
df.index = ['Mean', 'SD', 'n']
df

,1 cup or less/week,2-6 cups/week,1 cup/day,2-3 cups/day,4 or more cups/day
Mean,18.7,19.6,19.3,19.9,17.5
SD,21.1,25.5,22.5,22.0,22.0
n,12215.0,6617.0,17234.0,12290.0,2383.0


We will run an ANOVA test at the 5% significance level to see if there are any differences between the means of these groups.

$H_0 : \mu_1 = \mu_2 = \cdots = \mu_5$

$H_1 :$ At least one of $\mu_i$ for $i \in {1,2,3,4,5}$  is not equal.

where $\mu_i$ represent the mean amount of exercise in each group ordered from least to most coffee consumption.

We need to compute:

$ \bar{x} = \frac{1}{n}\sum_{i=1}^n x_i  $

$ SST = \sum_{i=1}^n (x_i - \bar{x})^2 $

$ SSG = \sum_{i=1}^5 (\bar{x_i} - \bar{x})$

$ SSE = \sum_{i=1}^5 (n_i-1)s_i^2 $

$ MSG = \frac{1}{4}SSG $

$ MSE = \frac{1}{n-5}SSE $

$ F = MSG/MSE $

In [154]:
# Compute x_bar first

n = df.iloc[2,:].sum()

def total(row):
    return row.iloc[0] * row.iloc[2]
    
x_bar = df.apply(total, axis=0).sum() / n

# SST is given to us in the question as we don't have the full data set.

SST = 25575327

# Now SSG

def SSG(row):
    return row.iloc[2] * (row.iloc[0]-x_bar)**2

SSG = df.apply(SSG, axis=0).sum()

# Now for SSE

def SSE(row):
    return row.iloc[1]**2 * (row.iloc[2]-1)

SSE = df.apply(SSE, axis=0).sum()

MSG = SSG/4

MSE = SSE/(df.iloc[2,:].sum()-5)

F = MSG/MSE
F

np.float64(8.452905882053816)

Having computed our F-score, we now compute the p-value:

In [160]:
p = stats.f(dfn=4, dfd=(n-4)).sf(F)
p

np.float64(8.185882702682308e-07)

Hence we have significant evidence to reject the null hypothesis that none of the means differ.

#### 7.47 Exercise

We are given the following data on food consumption whilst gaming vs not. Is there sufficient evidence of a difference in the mean quantity of food eaten between groups?

In [11]:
df = pd.DataFrame({'Gaming': [52.1, 45.1, 22], 'Control':[27.1, 26.4, 22]})
df.index = ['Mean', 'Standard Deviation', 'n']
df

,Gaming,Control
Mean,52.1,27.1
Standard Deviation,45.1,26.4
n,22.0,22.0


Let $\mu_G, \, \mu_C$ ne the mean food consumption in the gaming and control group respectively. Then, our hypothesese are:

$H_0 : \mu_G = \mu_C $

$H_1 : \mu_G \neq 0 \mu_C $

We will test this at the 5% significance level.

In [38]:
# Standard error
SE = np.sqrt(sum(df.iloc[1]**2/df.iloc[2]))

# Point estimate of difference in means
means = df.loc['Mean', 'Gaming'] - df.loc['Mean', 'Control']

# p-value
p = 2*stats.t(df=21, scale=SE).sf(means)
p

np.float64(0.035750822671415335)

Thus we have sufficient evidence to reject the null hypothesis here. There is evidence that gaming whilst eating leads to larger quantities of food being consumed than not gaming.

#### 7.53 Exercise

We want to detect a change of 1.0% with 80% power. Thus we need an overlap region of 80% between the two distributions. Under the null distribution, for a 95% hypothesis test, the upper portion of the critical region is any value larger than or equal to the following.

In [72]:
a=stats.norm.ppf(0.975)
a

np.float64(1.959963984540054)

For an alternative dsitribution which has an 80% overlap with the null distribution, we require

In [73]:
b=stats.norm.ppf(0.2)
b

np.float64(-0.8416212335729142)

We thus need the difference in means of the two distributions to be equal to

In [74]:
total = a+abs(b)
total

np.float64(2.8015852181129683)

standard errors. We thus compute the standard error.

In [75]:
SE_1 = 1/total
SE_1

np.float64(0.35694077536344176)

So we require a standard error of 0.357% to detect a 1% change at 80% power when testing at the 5% significance level.

In the first experiement, if a 95% confidence interval for a point estimate of 1.2% was (-0.2, 2.6), then the standard error in this experiment must have been given by:

In [76]:
SE_2 = ((2.6+0.2)/2)/(stats.norm.ppf(0.975)) 
SE_2

np.float64(0.7142988396945156)

If we wish to run another experiment where the standard error is 1.97 times smaller again, we require a new standard error of:

In [82]:
SE_3 = SE_1/1.97
SE_3

np.float64(0.1811882108443867)

We estimate the standard error by $\sqrt{\frac{s}{n}}$. Thus, if $n_3, \, n_1$ are the respective sizes of the samples and if we assume the standard deviation of the samples remains unchanged, we have

$n_3 = (1.97^2) n_1$

We thus need a sample size which is

In [85]:
1.97**2

3.8809

larger than the previous one.

Based on this, an experiment size of about 1.97% vs 1.97% would be more sensible to detect any significant differences.

#### 7.55 Exercise

We are given the sample average and sample standard deviation of 3.2 and 1.97 respectively and a sample size of 203.

With a sample size of 203, we would model the null distribution as a t-distribution with 202 degrees of freedom. Since $df>30$ we need only check for any extreme outliers. The histogram we are given in the text suggests there are no extreme outliers. We are told the sample is reasonably random so we will assume independence between observations holds. Our data should also be reasonably normally distributed which does not seem to be the case so we must proceed with caution in interpreting our findings.

In [92]:
# Load variables given
n = 203
s = 1.97
x_bar = 3.2

# Compute SE
SE = np.sqrt(s/n)

# Compute half-length of confidence interval
t = stats.t(df=202, scale = SE).ppf(0.95)

# Display the confidence interval in a readable format
(float(x_bar-t), float(x_bar+t))

(3.0372171374851606, 3.3627828625148397)